# Generate content for canvas

The process of working on Canvas to keep the videos, links, assignments, etc. has proven to be brittle. Further, once we are done testing the pages in `comp205`, we need to update all the links in canvas to point to `comp205-dist`.

`canvasgen.ipynb` is driven from a `canvas.csv` file and generates the html for each module that can be pasted into canvas directly. 

To update Canvas, first update `canvas.csv`, then run the notebook, then go to the Assignment in Canvas, click on Edit, Click on HTML Editor and paste the *a portion of the HTML* that `canvasgen` has generated.

To get the HTML for going against `comp205-dist`, change the `live` column in `canvas.csv` to `y` and run the notebook again. At the moment, the `y` option is broken!

Sometimes it is necessary to ignore a row in `canvas.csv` but still keep the other information in it around (so we can turn it on again later). To do this change the `live` column in `canvas.csv` to `x` and run the notebook again.

In [ ]:
"""
    # The CSV file should have these columns:
    # course_id,unit_int,unit_ext,module_int,module_ext,video,video_len,one_link,pdf_link,live
    'course_id': '10425',                       # Course ID in Canvas, (10425 for Fall 2019)
    'unit_ext': '02 Python Programming',        # External name of the Unit
    'unit_int': '02-python-programming',        # Internal name of the Unit
    'module_ext': '02-11 Rituals and Patterns', # External name of the Module
    'module_int': '02-11-patterns',             # internal name of the Module
    'one_link': '1053508',                      # Number of the .one file
                            # Retrieve link address of the .one file, it looks like
                            #   https://canvas.tufts.edu/files/1053508/download?download_frd=1
                            #   Pick out the number (1053508 in this case)
    'pdf_link': '1053509',                      # Number of the .pdf file
    'video': '1_w3cemhzj',
                            # Retrieve link address of the media file, it looks like
                            #   https://1813261-1.kaf.kaltura.com/media/t/1_w3cemhzj/133896931
                            #   Pick out the video identifier (1_w3cemhzj in this case)
    'video_len': '10:03',                       # Length of the video
    'live': 'n',                                # whether or not the generated links should point to a live site
                                                # y for yes, anything else for no
"""

import csv
import pprint
pp = pprint.PrettyPrinter(indent=4)

    # live_link was put in place by Prof. Couch in summer 2019.
    # test_link was put in place by Prof. Singh in autumn 2019.
    # For an explanation of test_link, see https://github.com/jupyterhub/jupyterhub/issues/2017

live_link = "git-pull?repo=https%3A%2F%2Fgithub.com%2Fprof-couch%2F{env}&amp;branch=master&amp;subPath={unit_int}%2F{module_int}%2F{module_int}.ipynb&amp;app=notebook"
test_link = "git-pull?repo=https%3A%2F%2Fgithub.com%2Fprof-couch%2F{env}&amp;branch=master&amp;subPath={unit_int}%2F{module_int}%2F{module_int}.ipynb&amp;app=notebook"
#test_link = "notebooks/comp205/{unit_int}/{module_int}/{module_int}.ipynb"
module_master = """
<!-- {module_ext}.     Live: {live}-->

<h2>Part 1: Watch the Video</h2>
<p><iframe style="width: 560px; height: 395px;" title="{module_ext} ({video_len})" 
    src="https://canvas.tufts.edu/courses/10425/external_tools/retrieve?display=borderless&amp;url=https%3A%2F%2F1813261-1.kaf.kaltura.com%2Fbrowseandembed%2Findex%2Fmedia%2Fentryid%2F{video}%2FshowDescription%2Ffalse%2FshowTitle%2Ffalse%2FshowTags%2Ffalse%2FshowDuration%2Ffalse%2FshowOwner%2Ffalse%2FshowUploadDate%2Ffalse%2FplayerSize%2F560x395%2FplayerSkin%2F39209821%2F" 
    width="560" height="395" allowfullscreen="allowfullscreen"
webkitallowfullscreen="webkitallowfullscreen" mozallowfullscreen="mozallowfullscreen" allow="autoplay *">
</iframe></p>
<h2>Part 2: Complete the Exercise</h2>
<ul>
<li>Exercise Link: <a href="https://jupyterhub.cs.tufts.edu/hub/user-redirect/{notebook_link}" target="_blank">{module_ext}</a></li>
</ul>
<h2>Resources</h2>
<ul>
<li><strong>Video Lecture Notes: 
<a class="instructure_file_link instructure_scribd_file" title="{module_int}.pdf" href="https://canvas.tufts.edu/courses/{course_id}/files/{pdf_link}/download?wrap=1" data-api-endpoint="https://canvas.tufts.edu/api/v1/courses/{course_id}/files/{pdf_link}" data-api-returntype="File">{module_int}.pdf</a> or 
<a class="instructure_file_link" title="{module_int}.one" href="https://canvas.tufts.edu/courses/{course_id}/files/{one_link}/download?wrap=1" data-api-endpoint="https://canvas.tufts.edu/api/v1/courses/{course_id}/files/{one_link}" data-api-returntype="File">{module_int}.one</a>.
<a class="instructure_file_link" title="{module_int}.one" href="https://canvas.tufts.edu/courses/{course_id}/files/{pdf_link}/download?wrap=1" data-api-endpoint="https://canvas.tufts.edu/api/v1/courses/{course_id}/files/{pdf_link}" data-api-returntype="File"></a></strong>
</li>
</ul>


"""

data=[]
reader = csv.DictReader(open('canvas.csv', 'r'), skipinitialspace = True)
for line in reader:
    line.update({fieldname: value.strip() for (fieldname, value) in line.items()})
    module_dict = dict(line)
    if not module_dict["live"].startswith("x"): # ignore csv lines where `live` is set to x.
        is_live = module_dict["live"].startswith("y")
        module_dict["env"] = 'comp205-dist' if is_live else 'comp205'
        module_dict["notebook_link"] = live_link.format_map(module_dict) if is_live else test_link.format_map(module_dict)
        #pp.pprint(module_dict)
        print(module_master.format_map(module_dict))